## Toronto One Address Shapefile Processing ##

#### Description ####

Due to large amount data processing needed for street addresses and the limitation of the Geopy API, I'm trying to use the Toronto address repository to process the street coordinates instead of Geopy API. 

It converts the shape file into df, and exports for later use. Also does a test for coordinates matching and merging.

Data source:
https://open.toronto.ca/dataset/address-points-municipal-toronto-one-address-repository/

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import shapefile

In [22]:
# Reading the shape file from Toronto One Address Repository

def read_shapefile(file_path):
    
    #read file
    sf = shapefile.Reader(file_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    
    #write to df
    toronto_addresses_df = pd.DataFrame(columns=fields, data=records)
    #toronto_addresses_df = toronto_addresses_df.assign(coords=shps)
    
    toronto_addresses_df['full_address'] = toronto_addresses_df['ADDRESS'] + ' ' + toronto_addresses_df['LFNAME']
    toronto_addresses_df['long_lat'] = toronto_addresses_df[['LONGITUDE','LATITUDE']].apply(tuple, axis=1)
    
    return toronto_addresses_df

toronto_addresses_file = 'geo_data/municipal-address-points-wgs84-latitude-longitude/ADDRESS_POINT_WGS84.shp'

toronto_addresses_df = read_shapefile(toronto_addresses_file)

In [24]:
# converting full_address to uppercase for easy matching with address in ticket data
toronto_addresses_df['full_address'] = toronto_addresses_df['full_address'].str.upper()
toronto_addresses_df.head()

,GEO_ID,LINK,MAINT_STAG,ADDRESS,LFNAME,LONUM,LONUMSUF,HINUM,HINUMSUF,ARC_SIDE,...,CLASS,NAME,X,Y,LONGITUDE,LATITUDE,MUN_NAME,WARD_NAME,full_address,long_lat
0,5729533,5729516,REGULAR,404,Lake Promenade,404,,0,,R,...,Land,,301526.896,4827441.250,-79.540536,43.586906,Etobicoke,Etobicoke-Lakeshore,404 LAKE PROMENADE,"(-79.5405358155, 43.5869056491)"
1,5729531,5729516,REGULAR,402,Lake Promenade,402,,0,,R,...,Land,,301539.058,4827450.035,-79.540385,43.586985,Etobicoke,Etobicoke-Lakeshore,402 LAKE PROMENADE,"(-79.5403852582, 43.586984775)"
2,5729535,5729516,REGULAR,407,Lake Promenade,407,,0,,L,...,Land,,301576.469,4827413.878,-79.539922,43.586659,Etobicoke,Etobicoke-Lakeshore,407 LAKE PROMENADE,"(-79.5399217575, 43.5866594714)"
3,5729534,5729516,REGULAR,405,Lake Promenade,405,,0,,L,...,Land,,301586.074,4827422.263,-79.539803,43.586735,Etobicoke,Etobicoke-Lakeshore,405 LAKE PROMENADE,"(-79.5398028653, 43.5867349937)"
4,5729532,5729516,REGULAR,403,Lake Promenade,403,,0,,L,...,Land,,301593.239,4827433.953,-79.539714,43.586840,Etobicoke,Etobicoke-Lakeshore,403 LAKE PROMENADE,"(-79.5397142074, 43.5868402558)"


In [16]:
toronto_addresses_df.to_csv('geo_data/geodata_toronto_addresses.csv')

In [47]:
#check if one address is can be found in toronto_addresses df
test_address = '202 DOVERCOURT RD'
#test_address = 'Ford St and St Clair Ave w' #doesn't really work for intersections

for i in range(len(toronto_addresses_df)):
    actual_address = toronto_addresses_df.loc[i, 'full_address']
    if test_address == actual_address:
        print('match found')
        print(toronto_addresses_df.loc[i, 'long_lat'])
        break
    else:
        pass

match found
(-79.4240340808, 43.6469600856)


In [103]:
# test dataframe
test_data = pd.read_csv('test_data.csv')
test_df = pd.DataFrame(test_data)
print('Number of rows and cols: ', str(test_df.shape))
test_df.head()

Number of rows and cols:  (49, 11)


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***71720,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,12,NR,202 DOVERCOURT RD,NaN,NaN,ON
1,***61115,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,19,W/S,GREAT WEST DR,S/O,DE JONG ST,ON
2,***61117,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,22,NR,3 DE JONG ST,NaN,NaN,ON
3,***61118,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,23,OPP,54 ZEZEL WAY,NaN,NaN,ON
4,***92520,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,24,E/S,FORD ST,S/O,ST CLAIR AVE W,ON


In [106]:
# keep the rows with real addresses, does not include intersections
for i in range(len(test_df)):
    test_df.loc[i, 'address_exist'] = test_df.loc[i, 'location2'][0].isdigit()

test_df = test_df[test_df['address_exist']]
test_df = test_df.reset_index(drop=True)
test_df = test_df.drop('address_exist', 1)
test_df.head()       

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***71720,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,12,NR,202 DOVERCOURT RD,NaN,NaN,ON
1,***61117,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,22,NR,3 DE JONG ST,NaN,NaN,ON
2,***61118,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,23,OPP,54 ZEZEL WAY,NaN,NaN,ON
3,***92328,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,28,NR,104 BOWIE AVE,NaN,NaN,ON
4,***92329,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,28,NR,103 BOWIE AVE,NaN,NaN,ON


In [67]:
# Print long_lat if a match is found --- this method is slow as hell
# for i in range(len(test_df)):
#     actual_address = test_df.loc[i, 'full_address']
#     print(actual_address)

#     for j in range(len(toronto_addresses_df)):
#         if actual_address == toronto_addresses_df.loc[j, 'full_address']:
#             #test_df.loc[i, 'long_lat'] = toronto_addresses_df.loc[j, 'long_lat']
#             print('match found')
#             print(toronto_addresses_df.loc[j, 'long_lat'])
#             break
#         else:
#             pass

202 DOVERCOURT RD
match found
(-79.4240340808, 43.6469600856)
3 DE JONG ST
match found
(-79.274909091, 43.7659469)
54 ZEZEL WAY
match found
(-79.275425111, 43.766147795)
104 BOWIE AVE
match found
(-79.4566773136, 43.696556066)
103 BOWIE AVE
match found
(-79.4561666741, 43.6962863485)
300 DOVERCOURT RD
match found
(-79.4251727529, 43.6499775099)
300 DOVERCOURT RD
match found
(-79.4251727529, 43.6499775099)
11 POLSON ST
match found
(-79.3547054387, 43.6410955976)
11 POLSON ST
match found
(-79.3547054387, 43.6410955976)
300 DOVERCOURT RD
match found
(-79.4251727529, 43.6499775099)
230 BICKNELL AVE
match found
(-79.476931405, 43.6883621432)
9 BAMBLETT DR
match found
(-79.2752416737, 43.7014765465)
102 MOBERLY AVE
match found
(-79.314116241, 43.684758967)
30 GRAND TRUNK CRES
match found
(-79.38280966, 43.6420814686)
30 GRAND TRUNK CRES
match found
(-79.38280966, 43.6420814686)
30 GRAND TRUNK CRES
match found
(-79.38280966, 43.6420814686)
36 ALFRESCO LWN
match found
(-79.294614988, 43.667864

In [108]:
# Try a table join to get the long_lat info
test_df = pd.merge(test_df, 
                   toronto_addresses_df[['full_address', 'MUN_NAME', 'WARD_NAME', 'long_lat']], 
                   left_on='location2',
                   right_on='full_address',
                   how='left')

In [109]:
test_df.head()

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province,full_address,MUN_NAME,WARD_NAME,long_lat
0,***71720,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,12,NR,202 DOVERCOURT RD,NaN,NaN,ON,202 DOVERCOURT RD,former Toronto,Davenport,"(-79.4240340808, 43.6469600856)"
1,***61117,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,22,NR,3 DE JONG ST,NaN,NaN,ON,3 DE JONG ST,Scarborough,Scarborough Centre,"(-79.274909091, 43.7659469)"
2,***61118,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,23,OPP,54 ZEZEL WAY,NaN,NaN,ON,54 ZEZEL WAY,Scarborough,Scarborough Centre,"(-79.275425111, 43.766147795)"
3,***92328,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,28,NR,104 BOWIE AVE,NaN,NaN,ON,104 BOWIE AVE,York,Eglinton-Lawrence,"(-79.4566773136, 43.696556066)"
4,***92329,20180101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,28,NR,103 BOWIE AVE,NaN,NaN,ON,103 BOWIE AVE,York,Eglinton-Lawrence,"(-79.4561666741, 43.6962863485)"
